In [ ]:
# CELL 0 – Imports and physical constants

import numpy as np
import matplotlib.pyplot as plt
from astropy.cosmology import FlatLambdaCDM
import astropy.units as u   # ← THIS is what was missing
from scipy.interpolate import griddata

# Physical constants
G   = 6.67430e-11         # m^3 kg^-1 s^-2
c   = 2.99792458e8        # m/s
M_sun = 1.98847e30        # kg
kpc = 3.085677581e19      # m

# (optional) cosmology object if you need it later
cosmo = FlatLambdaCDM(H0=70, Om0=0.3)

In [ ]:
# ================================================================
# CELL 1 — Lens configuration input block for HE 1104−1805
#      (observationally grounded / “real astronomy” baseline)
# ================================================================
# Change ONLY this cell for each lens.

# 1. Redshifts (lens & source)
# Lens redshift: z_d = 0.729 from VLT spectroscopy (Lidman et al. 2000)
# Source (quasar) redshift: z_s ≈ 2.322 (standard value for HE 1104-1805)
z_d = 0.729
z_s = 2.322

# 2. Angular-diameter distances using CELL 0 cosmology
D_d  = cosmo.angular_diameter_distance(z_d)            # [Mpc]
D_s  = cosmo.angular_diameter_distance(z_s)            # [Mpc]
D_ds = cosmo.angular_diameter_distance_z1z2(z_d, z_s)  # [Mpc]

# --- Convert to metres and build time-delay distance factor ---
D_d_m  = D_d.to(u.m).value
D_s_m  = D_s.to(u.m).value
D_ds_m = D_ds.to(u.m).value

# Time-delay distance factor: (1+z_d) * D_d * D_s / D_ds   [metres]
time_delay_distance_factor = (1.0 + z_d) * (D_d_m * D_s_m / D_ds_m)
print(f"(1+z_d) D_d D_s / D_ds = {time_delay_distance_factor:.3e} m")

# 3. Einstein radius (arcsec)
# HST / ground-based imaging give A–B separation ≈ 3.00".
# The lens galaxy lies about 1.00" from A along the A–B line,
# so A is at ~1.0" and B at ~2.0" from the lens.
# A natural choice for θ_E is the mean radius of the two images:
#   θ_E ≈ (R_A + R_B)/2 ≈ (1.00" + 2.00") / 2 ≈ 1.5"
theta_E_arcsec = 1.50

# 4. Lens mass ellipticity and orientation
# Early-type galaxy; shape parameters are poorly constrained in early IR data,
# so we adopt a mildly elliptical mass distribution aligned with the A–B axis.
axis_ratio_q       = 0.90     # b/a ~ 0.9 (mild ellipticity)
position_angle_phi = 0.0      # degrees East of North; 0° → major axis along +x

# 5. External shear
# Kochanek (2003) notes that HE 1104−1805 likely lies in an anomalously high
# tidal shear field. We reflect this with a relatively strong external shear.
gamma_ext     = 0.20          # shear amplitude (high, but observationally motivated)
phi_gamma_deg = 0.0           # shear PA aligned roughly with the A–B axis

# 6. External convergence (LOS mass-sheet)
# Deep IR imaging (Courbin et al. 1997/1998) finds no obvious overdensity of
# galaxies near the lens → environment not cluster-like. We therefore adopt
# a small, nonzero κ_env as a reasonable LOS contribution.
kappa_env = 0.03

# 7. Image positions (arcsec, lens-centered)
# HST PC1 imaging / deconvolution (Remy et al. 1998) find:
#   - A–B separation: θ_AB ≈ 3.00193"
#   - Lens galaxy G is ~1.0003" from A along the A–B line.
# If we place the lens at (0, 0) and align A–B along the x-axis:
#   R_A ≈ 1.0003"
#   R_B ≈ θ_AB − R_A ≈ 3.00193" − 1.0003" ≈ 2.0016"
# We take A to the East (+x) and B to the West (−x).
image_positions = {
    "A": ( +1.0003,  0.0000 ),
    "B": ( -2.0016,  0.0000 ),
}

# 8. MGE parameters (shape only)
# Survey-level 10-Gaussian template for an early-type lens galaxy.
# These amplitudes are arbitrary normalizations; the absolute scaling
# is fixed later by the θ_E normalization in CELL 3.
MGE_sigmas_arcsec = [
    0.06, 0.12, 0.20, 0.35, 0.55,
    0.80, 1.20, 1.80, 2.80, 4.50
]

MGE_amps = [
    9.0, 7.5, 6.0, 4.5, 3.0,
    2.0, 1.2, 0.6, 0.3, 0.10
]

(1+z_d) D_d D_s / D_ds = 1.483e+26 m


In [ ]:
# CELL 2 — Build grid & MGE model (does not change)

arcsec_to_rad = (np.pi/180)/3600

x_vals_arcsec = np.linspace(-5, 5, 500)
y_vals_arcsec = np.linspace(-5, 5, 500)
x_grid_arcsec, y_grid_arcsec = np.meshgrid(x_vals_arcsec, y_vals_arcsec)

def elliptical_gaussian(x, y, sigma, amp, q, phi_deg):
    phi = np.deg2rad(phi_deg)
    cosp, sinp = np.cos(phi), np.sin(phi)
    x_rot =  cosp*x + sinp*y
    y_rot = -sinp*x + cosp*y
    r2 = x_rot**2 + (y_rot/q)**2
    return amp*np.exp(-0.5*r2/sigma**2)

def psi_norm(x, y):
    psi = np.zeros_like(x)
    for sigma, amp in zip(MGE_sigmas_arcsec, MGE_amps):
        psi += elliptical_gaussian(
            x, y, sigma, amp,
            axis_ratio_q, position_angle_phi
        )
    return psi

psi_mass_norm = psi_norm(x_grid_arcsec, y_grid_arcsec)

In [ ]:
# CELL 3 — Normalize ψ via |∇ψ| = θ_E (does not change)

# Gradients
dpsi_dy_arcsec, dpsi_dx_arcsec = np.gradient(
    psi_mass_norm,
    y_vals_arcsec,
    x_vals_arcsec
)

# convert to radian deflection
dpsi_dx_rad = dpsi_dx_arcsec / arcsec_to_rad
dpsi_dy_rad = dpsi_dy_arcsec / arcsec_to_rad
alpha_mag = np.sqrt(dpsi_dx_rad**2 + dpsi_dy_rad**2)

# Einstein ring mask
theta_E_rad = theta_E_arcsec * arcsec_to_rad
r_grid = np.sqrt(x_grid_arcsec**2 + y_grid_arcsec**2)
ring_mask = np.abs(r_grid - theta_E_arcsec) <= 0.1

mean_defl_norm = alpha_mag[ring_mask].mean()
A_scale = theta_E_rad / mean_defl_norm

psi_mass_phys = psi_mass_norm * A_scale

In [ ]:
# CELL 4 — Add external shear (does not change)

phi_g = np.deg2rad(phi_gamma_deg)
cos2phi = np.cos(2*phi_g)
sin2phi = np.sin(2*phi_g)

x_rad = x_grid_arcsec * arcsec_to_rad
y_rad = y_grid_arcsec * arcsec_to_rad

psi_shear = 0.5*gamma_ext * (
    (x_rad**2 - y_rad**2)*cos2phi +
    2*x_rad*y_rad*sin2phi
)

psi_total = psi_mass_phys + psi_shear

In [ ]:
# ================================================================
# CELL 5 — Evaluate ψ_SFH at ALL image positions & compute all Δt
# ================================================================

def psi_at(theta_x, theta_y):
    """Interpolate ψ_SFH at an image position."""
    return griddata(
        (x_grid_arcsec.ravel(), y_grid_arcsec.ravel()),
        psi_total.ravel(),
        (theta_x, theta_y),
        method='cubic'
    )

# --- 1. Compute ψ for every image ---
psi_values = {}
for label, (x, y) in image_positions.items():
    psi_values[label] = psi_at(x, y)

print("ψ values at image positions:")
for k, v in psi_values.items():
    print(f"  ψ_{k} = {v:.6e}")
print()

# --- 2. Compute all pairwise delays ---
labels = list(image_positions.keys())
print("Pairwise SFH Time Delays:\n")

for i in range(len(labels)):
    for j in range(i+1, len(labels)):
        Li, Lj = labels[i], labels[j]
        psi_i = psi_values[Li]
        psi_j = psi_values[Lj]

        dpsi = psi_j - psi_i
        dt_raw = (dpsi * time_delay_distance_factor / c) / (24*3600)
        dt_corr = dt_raw / (1 - kappa_env)

        print(f"{Lj} – {Li}:")
        print(f"   Δψ = {dpsi:.6e}")
        print(f"   Raw SFH Δt = {dt_raw:.2f} days")
        print(f"   κ_env-corrected = {dt_corr:.2f} days\n")

ψ values at image positions:
  ψ_A = 6.475060e-11
  ψ_B = 2.898250e-11

Pairwise SFH Time Delays:

B – A:
   Δψ = -3.576810e-11
   Raw SFH Δt = -204.81 days
   κ_env-corrected = -211.14 days

